# Problem 2: EM for Bayesian Networks
### House Votes Dataset Analysis

In [2]:
from collections import defaultdict
from itertools import permutations, product

import numpy as np
import pandas as pd

## Data Preprocessing

This loads the `house-votes-84.data` dataset and preprocesses it:
1. **Load Data**: Reads the CSV file into a pandas DataFrame.
2. **Feature Selection**: Keeps only the first three columns (party affiliation and two votes).
3. **Column Renaming**: Labels columns as `party`, `vote1`, and `vote2` for clarity.
4. **Categorical Encoding**: Converts `democrat`/`republican` to `0`/`1`, `y`/`n` votes to `1`/`0`, and maps missing votes (`?`) to `NaN`.
5. **Train-Test Split**: Uses the first 300 rows for training and the remainder for testing.
This prepares the data for training Bayesian network models using the EM algorithm while handling missing values.

In [3]:
data = pd.read_csv('house-votes-84.data', header=None)
data = data.iloc[:, :3]
data.columns = ['party', 'vote1', 'vote2']

data['party'] = data['party'].map({'democrat': 0, 'republican': 1})
for col in ['vote1', 'vote2']:
    data[col] = data[col].map({'y': 1, 'n': 0, '?': np.nan})

train_data = data.iloc[:300]
test_data = data.iloc[300:]

In [4]:
print(train_data.to_markdown())

|     |   party |   vote1 |   vote2 |
|----:|--------:|--------:|--------:|
|   0 |       1 |       0 |       1 |
|   1 |       1 |       0 |       1 |
|   2 |       0 |     nan |       1 |
|   3 |       0 |       0 |       1 |
|   4 |       0 |       1 |       1 |
|   5 |       0 |       0 |       1 |
|   6 |       0 |       0 |       1 |
|   7 |       1 |       0 |       1 |
|   8 |       1 |       0 |       1 |
|   9 |       0 |       1 |       1 |
|  10 |       1 |       0 |       1 |
|  11 |       1 |       0 |       1 |
|  12 |       0 |       0 |       1 |
|  13 |       0 |       1 |       1 |
|  14 |       1 |       0 |       1 |
|  15 |       1 |       0 |       1 |
|  16 |       0 |       1 |       0 |
|  17 |       0 |       1 |     nan |
|  18 |       1 |       0 |       1 |
|  19 |       0 |       1 |       1 |
|  20 |       0 |       1 |       1 |
|  21 |       0 |       1 |       1 |
|  22 |       0 |       1 |     nan |
|  23 |       0 |       1 |       1 |
|  24 |     

In [5]:
print(test_data.to_markdown())

|     |   party |   vote1 |   vote2 |
|----:|--------:|--------:|--------:|
| 300 |       1 |       0 |       0 |
| 301 |       0 |       0 |       0 |
| 302 |       1 |       0 |       0 |
| 303 |       1 |       0 |       0 |
| 304 |       1 |       0 |       1 |
| 305 |       1 |       0 |       0 |
| 306 |       1 |       0 |       0 |
| 307 |       0 |       1 |       0 |
| 308 |       1 |       0 |       0 |
| 309 |       0 |       1 |       0 |
| 310 |       1 |       0 |       0 |
| 311 |       0 |       0 |       0 |
| 312 |       0 |       1 |       1 |
| 313 |       1 |       0 |       1 |
| 314 |       1 |       0 |       1 |
| 315 |       1 |       0 |       1 |
| 316 |       0 |       0 |       0 |
| 317 |       0 |       1 |       0 |
| 318 |       0 |       0 |       0 |
| 319 |       0 |       1 |       0 |
| 320 |       0 |       0 |       1 |
| 321 |       0 |       1 |       1 |
| 322 |       0 |       1 |       1 |
| 323 |       0 |       1 |       1 |
| 324 |     

This code generates all 6 possible **linear chain DAGs** for 3 variables (`0,1,2`):
1. **Permutations**: Creates every node order (e.g., `0→1→2`, `1→0→2`).
2. **Parent Assignment**: Each node (except first) gets the prior node as its parent.
3. **Output**: List of DAGs like `{0:[], 1:[0], 2:[1]}`, representing causal chains.

In [6]:
dags = []
for perm in permutations([0, 1, 2]):
    parents = {perm[0]: [], perm[1]: [perm[0]], perm[2]: [perm[1]]}
    dags.append(parents)

In [7]:
print(dags)

[{0: [], 1: [0], 2: [1]}, {0: [], 2: [0], 1: [2]}, {1: [], 0: [1], 2: [0]}, {1: [], 2: [1], 0: [2]}, {2: [], 0: [2], 1: [0]}, {2: [], 1: [2], 0: [1]}]


This code initializes **Conditional Probability Tables (CPTs)** for a Bayesian network:
1. **Seed Control**: Sets random seed for reproducibility.
2. **Root Nodes**: Initializes root node CPTs (`[P(0), P(1)]`) using a uniform Dirichlet distribution.
3. **Child Nodes**: Creates CPT arrays for all parent configurations (e.g., `2^len(parents)` rows), each row being a Dirichlet-sampled distribution.

In [8]:
def initialize_cpts(dag, seed):
    np.random.seed(seed)
    cpts = {}
    for node in range(3):
        parents = dag[node]
        if not parents:
            cpts[node] = np.random.dirichlet([1, 1])
        else:
            num_parent_configs = 2 ** len(parents)
            cpts[node] = np.random.dirichlet([1, 1], size=num_parent_configs)
    return cpts

This code initializes **expected counts** for EM's E-step:
1. **Structure**: Creates a dictionary `(node, parent_config, value) → count`.
2. **Parent Handling**: For nodes with parents, initializes counts for all `0/1` parent combinations.
3. **Root Nodes**: Initializes counts for root nodes without parents.
All counts start at `0.0` before accumulating probabilities.

In [9]:
def initialize_expected_counts(dag):
    expected_counts = defaultdict(float)
    for node in range(3):
        parents = dag[node]
        if parents:
            for p_combo in product([0, 1], repeat=len(parents)):
                expected_counts[(node, p_combo, 1)] = 0.0
                expected_counts[(node, p_combo, 0)] = 0.0
        else:
            expected_counts[(node, (), 1)] = 0.0
            expected_counts[(node, (), 0)] = 0.0
    return expected_counts

This code processes **complete samples** for the EM algorithm:
1. **Delta Tracking**: Counts `(node, parent_config, value)` occurrences.
2. **Log-Likelihood**: Computes sample probability using CPTs.
3. **Parent Handling**: Converts parent values to a CPT index via binary encoding.
4. **Probability Access**: Uses node value (`0/1`) to index CPT rows.

In [10]:
def process_complete_sample(sample_vals, dag, cpts):
    delta = defaultdict(float)
    log_likelihood = 0.0
    for node in range(3):
        parents = dag[node]
        val = int(sample_vals[node])

        if parents:
            p_vals = tuple(sample_vals[parents].astype(int))
            parent_config_idx = sum(p * (2 ** i) for i, p in enumerate(p_vals))
            prob_dist = cpts[node][parent_config_idx]
        else:
            prob_dist = cpts[node]

        p = prob_dist[val]
        log_likelihood += np.log(p)

        key = (node, p_vals if parents else (), val)
        delta[key] += 1
    return log_likelihood, delta

**Function Purpose**: Handles samples with missing values in the EM algorithm's E-step.
1. **Imputation**: Generates all possible `0/1` combinations for missing variables.
2. **Joint Probability**: Computes the probability of each imputed sample using CPTs.
3. **Normalization**: Weights each imputation by its probability (soft assignment).
4. **Delta Update**: Accumulates fractional counts (`(node, parent_config, value) → count`) for CPT updates.
5. **Log-Likelihood**: Computes the weighted log-probability of the sample.
**Key Step**: Uses binary encoding (`parent_config_index`) to access CPT rows for parent value combinations.

In [11]:
def process_incomplete_sample(sample_vals, dag, cpts, missing_vars):
    delta = defaultdict(float)
    log_likelihood = 0.0
    possible_combos = list(product([0, 1], repeat=len(missing_vars)))
    combo_probs, total_prob = [], 0.0

    for combo in possible_combos:
        imputed = sample_vals.copy()
        for i, var in enumerate(missing_vars):
            imputed[var] = combo[i]

        joint_prob = 1.0
        for node in range(3):
            parents = dag[node]
            val = int(imputed[node])

            if parents:
                p_vals = tuple(imputed[parents].astype(int))
                parent_config_index = sum([p * (2 ** i) for i, p in enumerate(p_vals)])
                prob_dist = cpts[node][parent_config_index]
            else:
                prob_dist = cpts[node]

            p = prob_dist[val]
            joint_prob *= p

        combo_probs.append(joint_prob)
        total_prob += joint_prob

    if total_prob == 0:
        return 0.0, delta

    for combo, joint_prob in zip(possible_combos, combo_probs):
        prob = joint_prob / total_prob
        imputed = sample_vals.copy()
        for i, var in enumerate(missing_vars):
            imputed[var] = combo[i]

        for node in range(3):
            parents = dag[node]
            val = int(imputed[node])

            if parents:
                p_vals = tuple(imputed[parents].astype(int))
            else:
                p_vals = ()

            key = (node, p_vals, val)
            delta[key] += prob

        log_likelihood += prob * np.log(joint_prob)

    return log_likelihood, delta

**Function Purpose**: Executes the **E-step** of the EM algorithm for Bayesian networks.
1. **Initialization**: Creates `expected_counts` to track node-parent-value occurrences.
2. **Sample Processing**: For each sample:
   - **Complete Data**: Directly computes counts/log-likelihood via `process_complete_sample`.
   - **Missing Data**: Imputes missing values probabilistically via `process_incomplete_sample`.
3. **Aggregation**: Accumulates log-likelihood and updates `expected_counts` for CPT estimation.
**Output**: Returns `expected_counts` (for M-step CPT updates) and total `log_likelihood` (for convergence checks).

In [12]:
def perform_e_step(dag, train_data, cpts):
    expected_counts = initialize_expected_counts(dag)
    log_likelihood = 0.0
    for _, sample in train_data.iterrows():
        sample_vals = sample.to_numpy()
        missing = [i for i, val in enumerate(sample_vals) if np.isnan(val)]
        if not missing:
            ll, delta = process_complete_sample(sample_vals, dag, cpts)
        else:
            ll, delta = process_incomplete_sample(sample_vals, dag, cpts, missing)
        log_likelihood += ll
        for key, value in delta.items():
            expected_counts[key] += value
    return expected_counts, log_likelihood

**Function Purpose**: Performs the **M-step** of the EM algorithm to update CPTs.
1. **Root Nodes**:
   - Computes `[P(node=0), P(node=1)]` from counts.
   - Uses Laplace smoothing (`[0.5, 0.5]`) if no data.
2. **Child Nodes**:
   - For each parent configuration (e.g., `(0,1)`):
     - Converts parent values to an index via binary encoding.
     - Computes `[P(node=0|parents), P(node=1|parents)]` from counts.
     - Applies Laplace smoothing if no data for the configuration.
3. **Output**: Returns updated CPTs as numpy arrays for efficient access.
**Key Feature**: Ensures valid probabilities even with missing data (no division by zero).

In [13]:
def perform_m_step(expected_counts, dag):
    new_cpts = {}
    for node in range(3):
        parents = dag[node]
        if not parents:
            count_0 = expected_counts.get((node, (), 0), 0)
            count_1 = expected_counts.get((node, (), 1), 0)
            total = count_0 + count_1
            if total == 0:
                new_cpt = np.array([0.5, 0.5])
            else:
                new_cpt = np.array([count_0 / total, count_1 / total])
            new_cpts[node] = new_cpt
        else:
            num_parent_configs = 2 ** len(parents)
            new_cpt = np.zeros((num_parent_configs, 2))
            for parent_config in product([0, 1], repeat=len(parents)):
                parent_config_idx = sum(p * (2 ** i) for i, p in enumerate(parent_config))
                count_0 = expected_counts.get((node, parent_config, 0), 0)
                count_1 = expected_counts.get((node, parent_config, 1), 0)
                total = count_0 + count_1
                if total == 0:
                    new_cpt[parent_config_idx] = [0.5, 0.5]
                else:
                    new_cpt[parent_config_idx] = [count_0 / total, count_1 / total]
            new_cpts[node] = new_cpt
    return new_cpts

**Function Purpose**: Implements the **EM algorithm** to learn CPTs for a Bayesian network.
1. **Initialization**: Generates random CPTs using `initialize_cpts`.
2. **EM Loop**:
   - **E-Step**: Computes expected counts and log-likelihood using current CPTs.
   - **M-Step**: Updates CPTs using expected counts.
   - **Convergence Check**: Stops if log-likelihood change is below `epsilon`.
3. **Termination**: Returns learned CPTs after convergence or reaching `max_iters`.
**Key Features**:
- Handles missing data via probabilistic imputation (E-step).
- Guarantees convergence by monitoring log-likelihood improvement.
- Seed ensures reproducible CPT initialization.

In [14]:
def learn_em(dag, train_data, max_iters=100, epsilon=1e-3, seed=None):
    cpts = initialize_cpts(dag, seed)
    prev_log_likelihood = -np.inf
    for _ in range(max_iters):
        expected_counts, log_likelihood = perform_e_step(dag, train_data, cpts)
        new_cpts = perform_m_step(expected_counts, dag)
        if np.abs(log_likelihood - prev_log_likelihood) < epsilon:
            break
        prev_log_likelihood = log_likelihood
        cpts = new_cpts
    return cpts

Train models for each DAG

In [15]:
models = []
for dag in dags:
    cpts = learn_em(dag, train_data)
    models.append((dag, cpts))

**Function Purpose**: Compares two sets of CPTs to check for convergence or equivalence.
1. **Node-wise Check**: Iterates through all nodes in the Bayesian network.
2. **Array Handling**: For array-based CPTs (child nodes), uses `np.allclose` to compare element-wise within `tol`.
3. **Scalar Handling**: For scalar CPTs (root nodes), checks absolute difference.
4. **Tolerance**: Allows slight numerical differences (e.g., `tol=0.01`).
**Returns**: `True` if all CPTs match within tolerance, `False` otherwise.

In [16]:
def compare_cpts(cpts1, cpts2, tol=1e-2):
    for node in cpts1:
        if isinstance(cpts1[node], np.ndarray):
            if not np.allclose(cpts1[node], cpts2[node], atol=tol):
                return False
        else:
            if abs(cpts1[node] - cpts2[node]) > tol:
                return False
    return True

**Code Purpose**: Tests if EM algorithm produces different models across runs.
1. **Multiple Runs**: Trains models on 20% masked data with different seeds.
2. **CPT Storage**: Saves learned CPTs for each run.
3. **CPT Comparison**: Checks if CPTs differ between runs using `compare_cpts`.
**Key Insights**:
- **Initialization Sensitivity**: Different seeds → different CPT initializations/missing data masks.
- **Local Optima**: EM may converge to different CPTs if likelihood has multiple peaks.
**Output**: Prints whether models differ (Yes/No), confirming EM's initialization dependence.

In [17]:
runs = []
for seed in [42, 99, 123, 7, 2023]:
    np.random.seed(seed)
    mask = np.random.rand(*train_data.shape) < 0.20
    train_data_masked = train_data.mask(mask)
    cpts = learn_em(dags[0], train_data_masked, seed=seed)
    runs.append(cpts)
    print(f"\n--- Run {seed} CPTs ---")
    for node in cpts:
        print(f"Node {node}: {cpts[node]}")

print("\nDo different EM runs produce different models?")
all_same = True
for i in range(len(runs)):
    for j in range(i + 1, len(runs)):
        if not compare_cpts(runs[i], runs[j]):
            all_same = False
            print(f"Run {i + 1} vs Run {j + 1}: Different CPTs")
if all_same:
    print("All runs produced identical models")
else:
    print("Conclusion: Yes, different runs yield different models")


--- Run 42 CPTs ---
Node 0: [0.62333333 0.37666667]
Node 1: [[0.39454685 0.60545315]
 [0.8125     0.1875    ]]
Node 2: [[0.4982726  0.5017274 ]
 [0.53159143 0.46840857]]

--- Run 99 CPTs ---
Node 0: [0.62333333 0.37666667]
Node 1: [[0.39454369 0.60545631]
 [0.8125     0.1875    ]]
Node 2: [[0.49858633 0.50141367]
 [0.53169426 0.46830574]]

--- Run 123 CPTs ---
Node 0: [0.62333333 0.37666667]
Node 1: [[0.39453932 0.60546068]
 [0.8125     0.1875    ]]
Node 2: [[0.49855713 0.50144287]
 [0.53158486 0.46841514]]

--- Run 7 CPTs ---
Node 0: [0.62333333 0.37666667]
Node 1: [[0.39454134 0.60545866]
 [0.8125     0.1875    ]]
Node 2: [[0.49847749 0.50152251]
 [0.53157879 0.46842121]]

--- Run 2023 CPTs ---
Node 0: [0.62333333 0.37666667]
Node 1: [[0.39454834 0.60545166]
 [0.8125     0.1875    ]]
Node 2: [[0.49825746 0.50174254]
 [0.53161258 0.46838742]]

Do different EM runs produce different models?
All runs produced identical models


**Code Purpose**: Evaluates Bayesian network models on test data to compute accuracy.
1. **Loop Over Models**: Tests each DAG structure (`dag`) and its learned CPTs (`cpts`).
2. **Test Sample Processing**: For each test sample:
   - Extracts observed votes (`vote1`, `vote2`) and true `party`.
   - Ignores samples with missing `party`.
3. **Probability Calculation**:
   - **Party Node (0)**: Uses parent configuration (if any) to fetch CPT probabilities `[P(0), P(1)]`.
   - **Vote Nodes (1,2)**: Multiplies probabilities of observed votes given their parent configurations.
4. **Prediction**: Predicts `party` using normalized probabilities (`prob_1 / total_prob > 0.5`).
5. **Accuracy**: Computes accuracy as `correct / total` and stores results.
**Key Mechanism**: Uses parent configuration indexing (`parent_config_idx`) to access CPTs efficiently.
**Output**: List of `(dag, accuracy)` pairs showing how well each DAG structure predicts party affiliation.

In [19]:
results = []
for dag, cpts in models:
    correct = 0
    total = 0
    for _, test_sample in test_data.iterrows():
        party = test_sample['party']
        if np.isnan(party):
            continue

        vote1 = test_sample['vote1']
        vote2 = test_sample['vote2']
        observed = {}
        if not np.isnan(vote1):
            observed[1] = int(vote1)
        if not np.isnan(vote2):
            observed[2] = int(vote2)

        prob_0 = 1.0
        prob_1 = 1.0

        node = 0
        parents = dag[node]
        if parents:
            p_vals = tuple([observed.get(p, 0) for p in parents])
            parent_config_idx = sum(p * (2 ** i) for i, p in enumerate(p_vals))
            prob_1 *= cpts[node][parent_config_idx][1]
            prob_0 *= cpts[node][parent_config_idx][0]
        else:
            prob_1 *= cpts[node][1]
            prob_0 *= cpts[node][0]

        for node in [1, 2]:
            if node not in observed:
                continue
            val = observed[node]
            parents = dag[node]
            if parents:
                p_vals = tuple([observed.get(p, 0) for p in parents])
                parent_config_idx = sum(p * (2 ** i) for i, p in enumerate(p_vals))
                p = cpts[node][parent_config_idx][val]
            else:
                p = cpts[node][val]

            prob_1 *= p
            prob_0 *= p

        total_prob = prob_0 + prob_1
        if total_prob == 0:
            predicted = 0
        else:
            predicted = 1 if (prob_1 / total_prob) > 0.5 else 0
        correct += (predicted == party)
        total += 1

    accuracy = correct / total if total else 0
    results.append((dag, accuracy))

DAG: {0: [], 1: [0], 2: [1]}, Accuracy: 0.6815
DAG: {0: [], 2: [0], 1: [2]}, Accuracy: 0.5926
DAG: {1: [], 0: [1], 2: [0]}, Accuracy: 0.6815
DAG: {1: [], 2: [1], 0: [2]}, Accuracy: 0.5926
DAG: {2: [], 0: [2], 1: [0]}, Accuracy: 0.6815
DAG: {2: [], 1: [2], 0: [1]}, Accuracy: 0.6815


In [230]:
for dag, acc in results:
    print(f"DAG: {dag}, Accuracy: {acc:.4f}")

DAG: {0: [], 1: [0], 2: [1]}, Accuracy: 0.5926
DAG: {0: [], 2: [0], 1: [2]}, Accuracy: 0.5926
DAG: {1: [], 0: [1], 2: [0]}, Accuracy: 0.6815
DAG: {1: [], 2: [1], 0: [2]}, Accuracy: 0.5926
DAG: {2: [], 0: [2], 1: [0]}, Accuracy: 0.5926
DAG: {2: [], 1: [2], 0: [1]}, Accuracy: 0.6815
